In [1]:
import os
import time
import numpy as np
import pandas as pd
import cv2

In [12]:
# adaptive thres + dilate-erode + open/close

def Hough_transform(filename,outimg, outmask):
    start_time=time.time()
    image = cv2.imread(filename)  
    print(" read time: %s seconds " % (time.time() - start_time)) 
    
    h, w, c = image.shape
    
    # resize large images
    if h>512:
        scale_percent = 30 # percent of original size
        w = int(image.shape[1] * scale_percent / 100)
        h = int(image.shape[0] * scale_percent / 100)
        dim = (w, h)
        # resize image
        image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    
    mindist = int(min(h,w) * 0.8)
    
    
    # keep copy of input image for final view
    output = image.copy()
    orig = image.copy()
    
    
    # convert image to grayscale for processing
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    
    # apply GuassianBlur to reduce noise. medianBlur is also added for smoothening, reducing noise.
    gray_blur = cv2.medianBlur(gray,9)
    #gray = cv2.medianBlur(gray,7)
    #gray = cv2.GaussianBlur(gray,(3,3),0)  # the tuple is the gaussian Kernel : controls the amount of blurring
    
    # threshold
    gray_thres = cv2.adaptiveThreshold(gray_blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,11,3.5)
    #ret3,gray_thres = cv2.threshold(gray_blur,50,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU) # Otsu
    
    
    # erosion-dilation
    #gray_e = cv2.erode(gray_thres,None,iterations = 3)
    #gray_d = cv2.dilate(gray_e,None,iterations = 3)
    # dilation-erosion
    gray_d = cv2.dilate(gray_thres,None,iterations = 1)
    gray_e = cv2.erode(gray_d,None,iterations = 1)
    #kernel1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(11,11))
    #gray_d = cv2.dilate(gray_thres,kernel1,iterations = 1)
    #gray_e = cv2.erode(gray_d,kernel1,iterations = 1)
    
    # hole filling
    kernel1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(11,11)) #(11,11)  (15,15)  (9,9)
    #close = cv2.morphologyEx(gray_e,cv2.MORPH_CLOSE,kernel1)
    # the closing operation adds white pixels; since the well boundary is black, it is breaking up,
    # need to use open operation instead
    close = cv2.morphologyEx(gray_e,cv2.MORPH_OPEN,kernel1)
    #close = cv2.morphologyEx(gray_thres,cv2.MORPH_OPEN,kernel1)
    
    # flood-fill
    #mask = np.zeros((h+2, w+2), np.uint8)
    #im_floodfill = close.copy()
    #cv2.floodFill(im_floodfill, mask, (0,0), 255);
    #im_floodfill_inv = cv2.bitwise_not(im_floodfill)
                                       
    
    # modify the parameters for houghtransform & retry    
    #circles = cv2.HoughCircles(gray_e, cv2.HOUGH_GRADIENT, 2, mindist)#, param1=30, param2=65, minRadius=0, maxRadius=500)
    circles = cv2.HoughCircles(close, cv2.HOUGH_GRADIENT, 2, mindist)#, param1=30, param2=65, minRadius=0, maxRadius=500)
    #circles = cv2.HoughCircles(im_floodfill_inv, cv2.HOUGH_GRADIENT, 2, mindist)#, param1=30, param2=65, minRadius=0, maxRadius=500)

    
    # create mask file
    #circle_image = np.zeros((h, w), dtype="uint8") #dtype=image.dtype)
    circle_image = np.zeros((h, w)) 
    #cv2.circle(circle_image, (int(w/2),int(h/2)), r, 255, -1)
    
    
    # ensure at least some circles were found
    if circles is not None:
        # convert the (x, y) coordinates and radius of the circles to integers
        circles = np.round(circles[0, :]).astype("int")
        
        # loop over the (x, y) coordinates and radius of the circles
        for (x, y, r) in circles:
            # draw the circle in the output image, then draw a rectangle
            # corresponding to the center of the circle
            cv2.circle(output, (x, y), r, (0, 255, 0), 4)
            cv2.rectangle(output, (x - 5, y - 5), (x + 5, y + 5), (0, 128, 255), -1)
            #cv2.circle(circle_image, (x,y), r, 255, -1)
            cv2.circle(circle_image, (x,y), r, 1, -1)
        
        # show the output image
        #cv2.imshow("output", np.hstack([image, output]))
        #cv2.waitKey(0)
        #showimage(output)
        #cv2.imwrite(outfile, output)   
        
    #else:
    #    cv2.imwrite(outfile, output)
    
    
    
    # replace the white pixels with 1, to avoid a 2ns round of encoding
    #result1 = circle_image.copy()
    #result1[circle_image > 0] = 1  # !=0
    #result1[circle_image <= 0] = 0
    #_, result1 = cv2.threshold(circle_image, thresh=180, maxval=255, type=cv2.THRESH_BINARY)
        
        
    # Make the grey scale image have three channels
    #circle_image3 = cv2.cvtColor(circle_image, cv2.COLOR_GRAY2BGR)
    
    
    # concatenate the images
    #numpy_horizontal_concat = np.concatenate((gray_thres3, gray_d3, gray_e3, output), axis=1)
    #numpy_horizontal_concat = np.concatenate((gray_thres3, close3, output), axis=1)
    #numpy_horizontal_concat = np.concatenate((orig, circle_image3, output), axis=1)
    cv2.imwrite(outimg, orig)
    cv2.imwrite(outmask, circle_image) #circle_image)  result1
    np.savetxt('mask_file.txt', circle_image)
    
    print(" Inference time: %s seconds " % (time.time() - start_time)) 


In [3]:
# generate masks with a subset of images

image_List=[]
with open(("sample_filelist_mask.txt"),'r') as fobj:
    for line in fobj:
        #print(line)
        image_List.append(line.rstrip("\n"))
fobj.close()

In [4]:
out1 = "C:/Users/data/Segment/images/"
out2 = "C:/Users/data/Segment/masks/"

#os.mkdir("C:/Users/data/Segment/")
#os.mkdir(out1)
#os.mkdir(out2)

https://forums.fast.ai/t/saving-binary-mask-opens-as-a-trinary-mask-why/51648

.jpg is a lossy format & may alter values.
use a non-lossy format for masks where exact pixel values are important.
Save the mask file as .png or .bmp

In [13]:
d = 0

for image in image_List:
    #print(d) #,image)
    outimg = out1 + "image_" + str(d) + ".jpg"
    outmask = out2 + "mask_image_" + str(d) + ".png"
    Hough_transform(image,outimg,outmask)   
    d = d+1

 read time: 0.05443716049194336 seconds 
 Inference time: 0.6363122463226318 seconds 
 read time: 0.016956090927124023 seconds 
 Inference time: 0.7933368682861328 seconds 
 read time: 0.021940946578979492 seconds 
 Inference time: 0.6163909435272217 seconds 
 read time: 0.022994279861450195 seconds 
 Inference time: 0.7430713176727295 seconds 
 read time: 0.01695394515991211 seconds 
 Inference time: 1.7688322067260742 seconds 
 read time: 0.020943880081176758 seconds 
 Inference time: 0.6120867729187012 seconds 
 read time: 0.016955137252807617 seconds 
 Inference time: 0.7292253971099854 seconds 
 read time: 0.023014068603515625 seconds 
 Inference time: 0.8021647930145264 seconds 
 read time: 0.04340815544128418 seconds 
 Inference time: 0.7913572788238525 seconds 
 read time: 0.02678203582763672 seconds 
 Inference time: 0.6666405200958252 seconds 
 read time: 0.02198314666748047 seconds 
 Inference time: 1.4545745849609375 seconds 
 read time: 0.028471946716308594 seconds 
 Infer